In [7]:
# Import required libraries for Reddit API data collection
import praw  # For making HTTP requests to Reddit API
import pandas as pd  # For data manipulation and analysis
import re
from datetime import datetime
import emoji
import string
import os
#from pysentimiento.preprocessing import preprocess_tweet
#from pysentimiento import create_analyzer

In [8]:
def get_reddit_instance():
    reddit = praw.Reddit(
        client_id = 'LDPs59MIc82xLXPftHQ3Sw',
        client_secret = "uZSSG2q3ueggUkUReIMzaKe8xnC6iw",
        password = 'Linyihao.041228',
        user_agent = "Comment Extraction (by SaltyfishQAQ)",
        username = 'Few-Strength-2343'
    )
    
    return reddit


def get_monthly_top_posts(reddit, subreddit_name, limit=10, time_filter='month'):
    count = 0
    # Fetch the subreddit
    subreddit = reddit.subreddit(subreddit_name)
    
    master_df = []
    all_submissions = []
    for submission in subreddit.top(time_filter=time_filter, limit=limit):
        single_submission = []
        # Collect submission information
        submission_info = {
            "submission_title": submission.title,
            "author": submission.author.name if submission.author else "deleted",
            "id": submission.id,
            "url": submission.url,
            "created_utc": submission.created_utc,
            "body": submission.selftext,  
            "type": "submission",
            "score": submission.score
        }
        master_df.append(submission_info)
        single_submission.append(submission_info)
        
        if submission.num_comments == 0:
            continue
        else:
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                # Skip MoreComments objects if any remain
                if isinstance(comment, praw.models.MoreComments):
                    continue
                
                comment_info = {
                    "submission_title": submission.title,
                    "author": comment.author.name if comment.author else "deleted",
                    "id": comment.id,
                    "url": f"https://www.reddit.com{comment.permalink}",
                    "created_utc": comment.created_utc,
                    "body": comment.body,
                    "type": "comment",
                    "score": comment.score
                }
                master_df.append(comment_info)
                single_submission.append(comment_info)
        
        all_submissions.append(single_submission)
        count += 1
        print(f"Processed submission {count}/{limit}: {submission.title}")
        
    master_df = pd.DataFrame(master_df).reset_index(drop=True)
        
    return master_df, all_submissions

In [9]:
reddit = get_reddit_instance()
master_df, all_submissions = get_monthly_top_posts(reddit, 'UofT', limit=1200, time_filter='year')


Processed submission 1/1200: [ Removed by Reddit ]
Processed submission 2/1200: Lady drives into cement in front of St George station 😭
Processed submission 3/1200: i walked 80 kilometers and 107000 steps for 24 hours straight to mcmaster university (from utsg)
Processed submission 4/1200: New required first year course just dropped, what do you guys think??
Processed submission 5/1200: Just graduated at Convocation with encampment present 
Processed submission 6/1200: I walked for 76 kilometers(and bussed 40 kilometers) from UofTSG to uWaterloo, with 2 strangers from the internet
Processed submission 7/1200: All that’s left of the encampment after todays events 
Processed submission 8/1200: Palestine protest going on at St George right now
Processed submission 9/1200: To those blokes who I wrongly accused of being mobsters, I am mortified and I owe you my life lmfao
Processed submission 10/1200: Students protesting near the encampment circle past 10 pm
Processed submission 11/1200: As

In [10]:
os.mkdir('individual_submissions_yearly') if not os.path.exists('individual_submissions_yearly') else None
for i, submission in enumerate(all_submissions):
    submission_df = pd.DataFrame(submission).reset_index(drop=True)
    submission_df.to_csv(f'individual_submissions_yearly/submission_{i}.csv', index=False)
    

In [11]:
master_df.shape

(30231, 8)

In [ ]:
# Data cleaning step

# for the selftexts: 
# lowercase all texts
# Delete spaces in front of and after the texts, keep the spaces in between
# Delete all special characters except for spaces
# Remove all error reading form the api. eg. error languange

# irelevant comments:
# Delete all comments that are not related to UofT: url not start from: https://www.reddit.com/r/UofT/comments/

# Time conveting:
# Convert time format in created_utc to readable format(20XX-XX-XX)
# utc format: scientific format for secs; 1.752931e+09 = 1,757,931,000 sec since Jan 1, 1970 UTC; 
# represent time stamp 2025-07-23 05:03:20

In [ ]:
def clean_text(text):
    """
    Remove all special characters from the text except for letters, numbers, and spaces.
    """
    return re.sub(r'[^\w\s]', '', text)


def convert_emoji_to_text(text, emoji_wrapper="emoji"):
    """
    Converts emoji in the text to descriptive text.
    
    This function uses emoji.demojize() to replace any emoji with their
    corresponding text (e.g., "😄" becomes ":smile:"). It then wraps the
    emoji description using the provided emoji_wrapper.
    
    Args:
        text (str): The original text containing emoji.
        emoji_wrapper (str): A string that will be used to wrap the emoji text.
        
    Returns:
        str: The text with emojis converted to wrapped text.
    """
    # Convert emojis to descriptive text (e.g., :smile:)
    demojized = emoji.demojize(text)
    # Define a wrapper string (e.g., " emoji ")
    wrapper = f" {emoji_wrapper} ".replace("  ", " ")
    # Replace the demojized emoji pattern :emoji_name: with the wrapped emoji_name
    # For example, ":smile:" becomes " emoji smile emoji "
    result = re.sub(r':([^:\s]+):', lambda m: wrapper + m.group(1) + wrapper, demojized)
    return result

def is_valid_word(word):
    """
    Returns True if the word is either:
    - Composed solely of punctuation
    - Contains at least one English letter (a-z or A-Z)
    """
    # Keep if word is only punctuation
    if all(ch in string.punctuation for ch in word):
        return True
    # Keep if the word contains at least one ASCII letter
    if re.search(r'[a-zA-Z]', word):
        return True
    return False

def filter_non_english(text):
    """
    Splits the text into words and filters out any word that is not an English word,
    an emoji, or punctuation.
    
    Args:
        text (str): The input text.
    
    Returns:
        str: The cleaned text.
    """
    words = text.split()
    filtered_words = [word for word in words if is_valid_word(word)]
    return ' '.join(filtered_words)

# 4. Convert created_utc to readable date format (YYYY-MM-DD)
def convert_utc_to_readable(utc_timestamp):
    """
    Convert UTC timestamp to a readable date format (YYYY-MM-DD).
    
    Args:
        utc_timestamp (float): UTC timestamp in seconds since epoch.
        
    Returns:
        str: Formatted date string or empty string if input is NaN.
    """
    
    try:
        # Handle NaN values
        if pd.isna(utc_timestamp):
            return ""
        
        # Convert to datetime object
        dt = datetime.fromtimestamp(utc_timestamp)
        
        # Format as YYYY-MM-DD HH:MM:SS
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    
    except (ValueError, TypeError, OSError):
        return ""
    

In [ ]:
# 1. Keep only posts with URLs starting with UofT comments
master_df = master_df[master_df['url'].str.startswith('https://www.reddit.com/r/UofT/comments/')].reset_index(drop=True).copy()

# 2. Clean selftext: lowercase, strip, remove special chars except spaces
master_df['body'] = master_df['body'].str.lower()
master_df['body'] = master_df['body'].str.strip()

# 3. Remove special characters except spaces
master_df['body'] = master_df['body'].apply(clean_text)

# 4. Converts emoji in the text to descriptive text.
master_df['body'] = master_df['body'].apply(convert_emoji_to_text)

# 5. Filter out non-English words
master_df['body'] = master_df['body'].apply(filter_non_english)

# 6. Convert created_utc to readable date format
master_df['created_utc'] = master_df['created_utc'].apply(convert_utc_to_readable)

# 7. Preprocess the selftext column using pysentimiento
# master_df['body'] = master_df['body'].apply(preprocess_tweet)

master_df.head()

In [ ]:


sentiment_analyzer = create_analyzer(task="sentiment", lang="en")

def sentiment_analysis(master_df, text_column='selftext'):
    """
    Iterates through each selftext in the DataFrame, applies emotion_analyzer,
    and adds two columns: sentiment_prediction and sentiment_prob.
    """
    senti_pred = []
    senti_prob = []
    
    for index, row in master_df.iterrows():
        text = row[text_column]
        
        if pd.isna(text) or text == "":
            senti_pred.append("NEU")
            senti_prob.append({"NEU": 0.34, "NEG": 0.33, "POS": 0.33})
        else:
            result = sentiment_analyzer.predict(text)
            senti_pred.append(result.output)
            senti_prob.append(result.probas)
    
    master_df['sentiment_prediction'] = senti_pred
    master_df['sentiment_prob'] = senti_prob
    
    return master_df

# Usage:
results_df = sentiment_analysis(master_df, text_column='body')
results_df.head()
# Export the DataFrame to an Excel file on your Desktop
results_df.to_excel("/Users/terrietaw/Desktop/uoft_sentiment_results.xlsx", index=False)